In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

df = pd.read_pickle('cleaned_df.pkl')

In [2]:
df.isnull().sum()

story        0
highlight    0
dtype: int64

In [4]:
# # load pretrained word2vec
# from gensim.test.utils import datapath
# from gensim.models import KeyedVectors
# path_of_downloaded_bin = "/Users/rohankilledar/Documents/MSc Artificial Intelligence/Natural Language Processing/labs/GoogleNews-vectors-negative300.bin"
# # #filename = 'GoogleNews-vectors-negative300.bin'
# word_vectors = KeyedVectors.load_word2vec_format(datapath(path_of_downloaded_bin), binary=True)

In [12]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

path_of_downloaded_files = "/Users/rohankilledar/Documents/MSc Artificial Intelligence/repos/Natural Language Processing/glove.6B.300d.txt"
filename = "glove.6B.300d.txt"

embedding_dim = 300

glove_file = datapath(path_of_downloaded_files)
word2vec_glove_file = get_tmpfile(filename)
glove2word2vec(glove_file, word2vec_glove_file)
word_vectors = KeyedVectors.load_word2vec_format(word2vec_glove_file)

<ipython-input-12-0f6affb87e2b>:10: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_file, word2vec_glove_file)


In [6]:
word_vectors[0].shape

(300,)

In [7]:
print('loaded %s word vectors from %s.' % (len(word_vectors.key_to_index),filename ))

loaded 3000000 word vectors from glove.6B.100d.txt.


In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test,y_train,y_test = train_test_split(df['story'].tolist(),df['highlight'].tolist(),test_size=0.3,random_state=2501,shuffle=True)

In [20]:
word_corpus = [] 
from nltk.tokenize import word_tokenize
for articles in tqdm(x_train):
    word_corpus.extend(word_tokenize(articles))
print(len(word_corpus))
unique_word_corpus = set(word_corpus)
print(len(unique_word_corpus))


100%|██████████| 64805/64805 [05:53<00:00, 183.33it/s]


47561196
215152


In [21]:

inter_word = set(word_vectors.key_to_index).intersection(unique_word_corpus)
print(len(inter_word))

145464


In [22]:
print("The number of words that are present in both glove vectors and our corpus are {} which \
is nearly {}% ".format(len(inter_word), np.round((float(len(inter_word))/len(unique_word_corpus))
*100)))


The number of words that are present in both glove vectors and our corpus are 145464 which is nearly 68.0% 


In [23]:
unique_word_corpus.difference(set(word_vectors.key_to_index))

{'knegt',
 '1401.',
 'hink',
 'illarionovich',
 'whc.unesco.org',
 'ourcq',
 'phegley',
 'adrb',
 'eligendo',
 'zhabdrung',
 'djkokovic',
 'refa',
 'equivocates',
 'knurr',
 'klinsy',
 'wippert',
 'fleurival',
 'contructor',
 'heimans',
 'sippie',
 'oyefeso',
 'ministiry',
 'eddmenson',
 'gurewardher',
 'pnemonia',
 '758.4',
 'antiscorbutics',
 'john.smith',
 'egality',
 'heinekens',
 'osobien',
 'isteve',
 'broncosports.com',
 'abdelmaseeh',
 'whbl',
 'kawekumnerd',
 '41,363',
 'ramineh',
 'causes.com',
 '68,792,500',
 '62,500.',
 'lakepowell.com',
 'mapness',
 'zablah',
 'dicari',
 'to1970s',
 'frietas',
 'abdelssalam',
 'shirine',
 'kalanick',
 'yovanna',
 'wouldisastrous',
 '602m',
 'hirschy',
 'indictator',
 'tufegdzic',
 '16,380',
 '51f',
 'rawker',
 'internatioal',
 'irva',
 'bernerd',
 'soei',
 'cesbron',
 'ruyati',
 'sermilik',
 'presente.org',
 'mrporter.com',
 'actuals',
 'corianton',
 'handicappedpets.com',
 'barrando',
 'qaracha',
 '5,202',
 '7,225',
 'isurvivor',
 '1,700k

68% seems reseaonable as most of the words out of the word_vocab are either number or names.

In [101]:
words = [] 
word2indx = {}

with open(path_of_downloaded_files) as f:
    for indx,l in enumerate(f):
        line = l.split()
        word = line[0]
        words.append(word)
        word2indx[word] = indx

# prewb = len(words)

# for indx,word in enumerate(unique_word_corpus):
#     words.append(word)
#     word2indx[word] = prewb+indx+1



In [102]:
print(len(word_vectors))
print(len(unique_word_corpus))
print(len(words))


400000
215152
400000


In [99]:
import pickle

# save model and other necessary modules
all_info_want_to_save = {
    'words': words,
    'word2indx': word2indx
}

with open("vocab.pkl","wb") as save_path:
    pickle.dump(all_info_want_to_save, save_path)




In [103]:
glove = {w: word_vectors[word2indx[w]] for w in words}

In [105]:
embedding_dim =300

In [106]:
matrix_len = len(unique_word_corpus)

weight_matrix  = np.zeros((matrix_len, embedding_dim))
words_found = 0

for indx, word in enumerate(unique_word_corpus):
    try:
        weight_matrix[indx] = glove[word]
        words_found +=1
    except KeyError:
        weight_matrix[indx] = np.random.normal(scale=0.6 , size= (embedding_dim,))

In [107]:
print(words_found)

145464


In [108]:
def create_emb_layer(weight_matrix, non_trainalbe = False):
    num_embeddings, embedding_dim = weight_matrix.size()
    emb_layer = nn.Embedding(num_embeddings,embedding_dim)
    emb_layer.load_state_dict({'weight': weight_matrix})
    if non_trainalbe:
        emb_layer.weight.requires_grad = False
    return emb_layer,num_embeddings,embedding_dim

    

In [109]:
class Encoder(nn.Module):
  def __init__(self, input_size, hidden_size,weight_matrix, bidirectional = True):
    super(Encoder, self).__init__()
    self.hidden_size = hidden_size
    self.input_size = input_size
    self.bidirectional = bidirectional
    self.embedding, self.num_embeddings, embedding_dim = create_emb_layer(weight_matrix,True)
    
    self.lstm = nn.LSTM(input_size, hidden_size, bidirectional = bidirectional)
  
  def forward(self, inputs, hidden):
    inputs = self.embedding(inputs)
    output, hidden = self.lstm(inputs.view(1, 1, self.input_size), hidden)
    return output, hidden
    
  def init_hidden(self):
    return (torch.zeros(1 + int(self.bidirectional), 1, self.hidden_size),
      torch.zeros(1 + int(self.bidirectional), 1, self.hidden_size))

In [ ]:
class AttentionDecoder(nn.Module):
  
  def __init__(self, hidden_size, output_size, vocab_size):
    super(AttentionDecoder, self).__init__()
    self.hidden_size = hidden_size
    self.output_size = output_size
    
    self.attn = nn.Linear(hidden_size + output_size, 1)
    self.lstm = nn.LSTM(hidden_size + vocab_size, output_size) #if we are using embedding hidden_size should be added with embedding of vocab size
    self.final = nn.Linear(output_size, vocab_size)
  
  def init_hidden(self):
    return (torch.zeros(1, 1, self.output_size),
      torch.zeros(1, 1, self.output_size))
  
  def forward(self, decoder_hidden, encoder_outputs, input):
    
    weights = []
    for i in range(len(encoder_outputs)):
      print(decoder_hidden[0][0].shape)
      print(encoder_outputs[0].shape)
      weights.append(self.attn(torch.cat((decoder_hidden[0][0], 
                                          encoder_outputs[i]), dim = 1)))
    normalized_weights = F.softmax(torch.cat(weights, 1), 1)
    
    attn_applied = torch.bmm(normalized_weights.unsqueeze(1),
                             encoder_outputs.view(1, -1, self.hidden_size))
    
    input_lstm = torch.cat((attn_applied[0], input[0]), dim = 1) #if we are using embedding, use embedding of input here instead
    
    output, hidden = self.lstm(input_lstm.unsqueeze(0), decoder_hidden)
    
    output = self.final(output[0])
    
    return output, hidden, normalized_weights
  

In [37]:
#Loading our Glove Model 
embeddings_index = dict()
f = open(path_of_downloaded_files)
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [39]:

words_corpus_source_train = {}
words_glove = set(embeddings_index.keys())
for i in unique_word_corpus:
  if i in words_glove:
    words_corpus_source_train[i] = embeddings_index[i]
print("word 2 vec length", len(words_corpus_source_train))

word 2 vec length 145464


In [72]:
from collections import Counter
from torchtext.vocab import Vocab
from torchtext.data.utils import get_tokenizer

tokenizer = get_tokenizer('basic_english')

def create_vocab(data):
    counter = Counter()
    for line in data:
        counter.update(tokenizer(line))
    vocab = Vocab(counter)
    print('vocab len: '+ str(len(vocab)))
    return vocab

vocab = create_vocab(x_train)





vocab len: 201650


torchtext.vocab.vocab.Vocab

In [70]:
embedding_dim = 300
embedding_matrix =np.zeros(len(unique_word_corpus)+1,embedding_dim)
for word in unique_word_corpus:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[]

True

In [48]:

from keras.preprocessing.text import Tokenizer 
x_tokenizer = Tokenizer()
x_tokenizer.fit_on_texts(list(x_train))


ModuleNotFoundError: No module named 'tensorflow'

In [45]:
from nltk.tokenize import word_tokenize,sent_tokenize
x_tokenizer = sent_tokenize
word_index = x_tokenizer.word_index

embedding_matrix = np.zeros((len(word_index)+1, 300))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


AttributeError: 'function' object has no attribute 'word_index'

In [35]:
import numpy as np
def vectorize_sent(word_vectors, sent):
    word_vecs = []
    word_list = []
    for token in sent.split():
        if token not in word_vectors: continue # if a word doesn't have an embedding, skip
        
        word_vecs.append(word_vectors[token]) 
        word_list.append(token)# otherwise, obtain the word embedding
    # print(np.array(word_vecs).shape)
   # return np.mean(np.array(word_vecs),axis=0) # compute the average embedding
    return np.array(word_vecs), word_list

senten = "Hey, how are you doing today? I went to St. Andrews this weekend. It's a beautiful place"
print(len(senten.split()))
sent_vec,word_list = vectorize_sent(word_vectors,senten)
print(sent_vec)
print(word_list)
print(sent_vec.shape[0])

17
[[-0.2852    -0.013883   0.31607   ... -0.10859   -0.14354    0.1485   ]
 [-0.23589    0.3831     0.10834   ... -0.79189   -0.08596   -0.1466   ]
 [-0.28427    0.047977  -0.15062   ... -0.090071   0.016922   0.29278  ]
 ...
 [-0.29712    0.094049  -0.096662  ...  0.059717  -0.22853    0.29602  ]
 [-0.23852   -0.33704   -0.26531   ...  0.32891    0.037239   0.23779  ]
 [ 0.080622   0.13208   -0.0047645 ... -0.063376  -0.46961    0.40538  ]]
['how', 'are', 'you', 'doing', 'went', 'to', 'this', 'a', 'beautiful', 'place']
10


In [50]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

vocab_file = "data/vocab"

embedding_dim = 128
MAX_LEN_STORY = 300
hidden_dim = 128
voc_size = 20000
MAX_LEN_HIGHLIGHT = 100

NUM_EPOCHS=100
batch_size=16
lr = 0.15

## tokens variable
PAD_TOKEN = '[PAD]'
UNKNOWN_TOKEN = '[UNK]'
START_TOKEN = '[START]'
STOP_TOKEN = '[STOP]'
SENTENCE_START = '<s>' # start sentence in highlight
SENTENCE_END = '</s>' # end sentence in highlight


In [54]:
optimizer = torch.optim.Adagrad(model.parameters(), lr=lr)
loss_function = torch.nn.NLLLoss()

NameError: name 'model' is not defined

In [53]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
#from vars import *
# from dataclasses import dataclass,field

# @dataclass(frozen = True, order = True)
# class Encoder(nn.Module):
#     num_layer : int = 1
#     embeddings : int = nn.Embedding(num_embeddings= voc_size, embedding_dim=hidden_dim)
#     lstm = nn.LSTM(input_size = hidden_dim,hidden_size = hidden_dim, num_layer =1, bidrectional = True, batch_first = True)




class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.num_layer = 1
        self.embeddings = nn.Embedding(
            num_embeddings=voc_size,
            embedding_dim=hidden_dim
        )
        self.lstm = nn.LSTM(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            num_layers=1,
            bidirectional=True,
            batch_first=True
        )

    def init_hidden(self):
        return (torch.zeros(2 * self.num_layer, batch_size, hidden_dim, dtype=torch.float, device=device),
                torch.zeros(2 * self.num_layer, batch_size, hidden_dim, dtype=torch.float, device=device))

    def forward(self, input, state):
        embedded = self.embeddings(input)
        output, (hidden, cell) = self.lstm(embedded, state)
        return output, (hidden, cell)


class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.embeddings = nn.Embedding(
            num_embeddings=voc_size,
            embedding_dim=hidden_dim
        )
        self.lstm = nn.LSTM(
            input_size=hidden_dim,
            hidden_size=hidden_dim * 2,
            num_layers=1,
            bidirectional=False,
            batch_first=True
        )
        self.fc = nn.Linear(
            in_features=hidden_dim * 2,
            out_features=voc_size
        )
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, cell):
        embedded = self.embeddings(input).unsqueeze(1)
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        output = self.fc(output.squeeze(1))
        output = self.logsoftmax(output)
        return output, (hidden, cell)


class AttnDecoder(nn.Module):
    def __init__(self):
        super(AttnDecoder, self).__init__()
        self.embeddings = nn.Embedding(
            num_embeddings=voc_size,
            embedding_dim=hidden_dim
        )
        self.lstm = nn.LSTM(
            input_size=hidden_dim,
            hidden_size=hidden_dim * 2,
            num_layers=1,
            bidirectional=False,
            batch_first=True
        )
        # attention layer (we use the definition in the paper hence tanh)
        self.W = nn.Sequential(
            nn.Linear(hidden_dim * 4, hidden_dim * 2),
            nn.Tanh(),
            nn.Linear(hidden_dim * 2, 1, bias=False)
        )
        self.fc = nn.Linear(
            in_features=hidden_dim * 4,
            out_features=voc_size
        )
        self.W_gen_sig = nn.Sequential(
            nn.Linear(hidden_dim * 2 + hidden_dim * 2 + hidden_dim, 1),
            nn.Sigmoid()
        )
        self.softmax = nn.Softmax(dim=1)
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, input, decoder_hidden, cell, encoder_outputs, story_extended, extra_zeros):
        (b, t_k, n) = encoder_outputs.shape
        embedded = self.embeddings(input).unsqueeze(1)  # batch * hidden_dim

        _, (decoder_hidden, cell) = self.lstm(embedded, (decoder_hidden, cell))

        decoder_hidden_expanded = decoder_hidden.permute(1, 0, 2)  # batch_size * hidden_dim
        decoder_hidden_expanded = decoder_hidden_expanded.expand(b, t_k,
                                                                 n).contiguous()  # batch_size * seq_len * hidden_dim

        att_features = torch.cat((decoder_hidden_expanded, encoder_outputs), 2)  # batch_size * seq_lens * 2*hidden_dim
        e_t = self.W(att_features).squeeze(2)  # batch_size * seq_lens
        a_t = self.softmax(e_t)  # batch_size * seq_lens

        a_applied = torch.bmm(a_t.unsqueeze(1), encoder_outputs).squeeze(1)

        s_t_h_t = torch.cat((decoder_hidden.squeeze(0), a_applied), 1)  # batch_size * 2*hidden_dim
        p_vocab = self.softmax(self.fc(s_t_h_t))

        cat_gen = torch.cat((a_applied, decoder_hidden.squeeze(0), embedded.squeeze(1)), 1)

        p_gen = self.W_gen_sig(cat_gen).clamp(min=1e-8)
        p_vocab = p_vocab * p_gen
        a_t_p_gen = (1 - p_gen) * a_t

        p_vocab_cat = torch.cat((p_vocab, extra_zeros), 1)
        output = p_vocab_cat.scatter_add(1, story_extended, a_t_p_gen).clamp(min=1e-8)
        return torch.log(output), (decoder_hidden, cell)


In [26]:
from torch.autograd import Variable
import torch
x_train_tensor = Variable(torch.Tensor(x_train))

ValueError: too many dimensions 'str'

In [39]:
# Baseline model with linear fully connected network and functionality to add dropout rate
#default activation function set to sigmoid and dropout rate to 0 making no difference to the NN
import torch.nn as nn
import torch

class BaselineModel(nn.Module):
    def __init__(self,act_fn=nn.Sigmoid(),dropout_rate=0):
        super(BaselineModel, self).__init__()
        
        self.layers = nn.Sequential(
        nn.Flatten(),
        nn.Linear(784, 1000),
        act_fn,
        nn.Dropout(dropout_rate),
        #nn.Sigmoid(),
        nn.Linear(1000, 784),
        act_fn,
        #nn.Sigmoid(),
        nn.Linear(784, 10)
        
       )
    def forward(self, x):
      x =  self.layers(x)
      return x
   


100%|██████████| 93626/93626 [00:05<00:00, 16774.15it/s]


all the words in the training corpus 61270167
unique words in the training corpus 349125
The number of words that are present in both glove vectors and our corpus are 88176 which is nearly 25.0% 
